In [14]:
class Car:
    '''
    A class to represent cars on a road and simulate traffic
    '''

    def __init__(self, vmax, dmin, initial_position, initial_speed):
        '''
        Constructor for the car class.

        Args:
        vmax (float): Maximum speed of the car.
        dmin (float): Minimum safe distance to the front car.
        initial_position (float): Initial position of the car on the road.
        initial_speed (float): Initial speed of the car.
        '''
        self.vmax = vmax
        self.dmin = dmin
        self.x = initial_position
        self.v = initial_speed
        self.dt = 1/3600  

    def update_velocity(self, front_car):
        '''
        Update the velocity of the car based on the position of the front car.

        Args:
        front_car (Car): The car in front of the current car.
        '''
        if front_car is not None:
            distance_to_front_car = front_car.x - self.dmin - self.x
            self.v = max(min(distance_to_front_car / self.dt, self.vmax), 0)  
        else:
            self.v = self.vmax

    def update_position(self):
        '''
        Update the position of the car based on its velocity.
        '''
        self.x += self.dt * self.v

initial_positions = [2.05, 2.04, 2.03, 2.02, 2.0]
initial_speeds = [60, 70, 80, 90, 105]

cars = [Car(105, 0.001, initial_positions[i], initial_speeds[i]) for i in range(5)]

for t in range(10):
    for i in range(4, -1, -1):
        front_car = cars[i - 1] if i > 0 else None
        # prev_position = cars[i - 1].x if i > 0 else cars[i].x
        cars[i].update_velocity(front_car)
        cars[i].update_position()
        print("Time: {}s - Car {} - Position: {:.2f} km - Speed: {:.2f} km/h".format(t + 1, i + 1, cars[i].x, cars[i].v))

Time: 1s - Car 5 - Position: 2.02 km - Speed: 68.40 km/h
Time: 1s - Car 4 - Position: 2.03 km - Speed: 32.40 km/h
Time: 1s - Car 3 - Position: 2.04 km - Speed: 32.40 km/h
Time: 1s - Car 2 - Position: 2.05 km - Speed: 32.40 km/h
Time: 1s - Car 1 - Position: 2.08 km - Speed: 105.00 km/h
Time: 2s - Car 5 - Position: 2.03 km - Speed: 32.40 km/h
Time: 2s - Car 4 - Position: 2.04 km - Speed: 32.40 km/h
Time: 2s - Car 3 - Position: 2.05 km - Speed: 32.40 km/h
Time: 2s - Car 2 - Position: 2.08 km - Speed: 105.00 km/h
Time: 2s - Car 1 - Position: 2.11 km - Speed: 105.00 km/h
Time: 3s - Car 5 - Position: 2.04 km - Speed: 32.40 km/h
Time: 3s - Car 4 - Position: 2.05 km - Speed: 32.40 km/h
Time: 3s - Car 3 - Position: 2.08 km - Speed: 105.00 km/h
Time: 3s - Car 2 - Position: 2.11 km - Speed: 105.00 km/h
Time: 3s - Car 1 - Position: 2.14 km - Speed: 105.00 km/h
Time: 4s - Car 5 - Position: 2.05 km - Speed: 32.40 km/h
Time: 4s - Car 4 - Position: 2.08 km - Speed: 105.00 km/h
Time: 4s - Car 3 - Posit

In [4]:
import unittest

class TestCarSimulation(unittest.TestCase):
    '''
    A class to test the simulation of cars on a road.
    '''

    def test_car_simulation_scenario(self):
        '''
        Test the simulation of cars on a road.
        '''
        vmax = 105
        dmin = 0.001

        initial_positions = [2.05, 2.04, 2.03, 2.02, 2.0]
        initial_speeds = [60, 70, 80, 90, 105]

        cars = [Car(vmax, dmin, initial_positions[i], initial_speeds[i]) for i in range(5)]

        for t in range(10):
            for i in range(4, -1, -1):
                front_car = cars[i - 1] if i > 0 else None
                prev_position = cars[i - 1].x if i > 0 else cars[i].x
                cars[i].update_velocity(front_car)
                cars[i].update_position()

                self.assertGreater(cars[i].x, 0)  #Check if position is always greater than 0
                self.assertLessEqual(cars[i].v, cars[i].vmax)  #Check if velocity never exceeds vmax

                #Check if the car at position i is within dmin distance from the front car
                if i > 0:
                    self.assertGreaterEqual(cars[i - 1].x + dmin, cars[i].x)

                self.assertGreaterEqual(cars[i].v, 0)  #Check that no car has a negative speed


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [19]:
class Road:
    '''
    A class representing a road and managing the traffic simulation.
    '''
    def __init__(self, length, initial_cars=[]):
        '''
        Constructor for the Road class.

        Args:
        length (float): Length of the road.
        initial_cars (list): List of initial cars on the road.
        '''
        self.length = length
        self.cars = initial_cars

    def add_car(self, initial_speed):
        '''
        Add a new car to the end of the road with a given initial speed.

        Args:
        initial_speed (float): Initial speed of the new car.
        '''
        new_car = Car(105, 0.001, 0, initial_speed)
        self.cars.append(new_car)

    def update_cars(self, dt):
        '''
        Update the positions and velocities of all cars on the road.

        Args:
        dt (float): Time step for the simulation.
        '''
        exited_cars_count = 0

        for i in range(len(self.cars) - 1, -1, -1):
            front_car = self.cars[i - 1] if i > 0 else None
            self.cars[i].update_velocity(front_car)
            self.cars[i].update_position()

            # Remove cars that have exited the road
            if self.cars[i].x >= self.length:
                self.cars.pop(i)
                exited_cars_count += 1

        if exited_cars_count > 0:
            print("{} car(s) exited the road.".format(exited_cars_count))

        return exited_cars_count

    def simulate(self, duration):
        '''
        Simulate the traffic on the road for a given duration.

        Args:
        duration (int): Duration of the simulation in seconds.
        '''
        for t in range(11):  # 10 iterations
            probability = random.random()

            # With a probability of 0.5, add a new car with a random initial speed
            if probability < 0.5:
                initial_speed = random.uniform(50, 120)
                self.add_car(initial_speed)
                print("New car added at time {}s with speed {}.".format(round(t, 3), round(initial_speed, 3)))

            # Update the positions and velocities of existing cars
            exited_count = self.update_cars(dt=1)

            # Print the current state
            self.print_state(t, exited_count)

    def print_state(self, time, exited_count):
        '''
        Print the current state of the road.

        Args:
        time (int): Current time in the simulation.
        exited_count (int): Number of cars exited the road in the current iteration.
        '''
        remaining_cars = [car for car in self.cars if car.x >= 0 and car.x <= self.length]
        print("Time: {}s - Number of Cars: {} - Exited Cars: {} - Car Positions: {} - Car Speeds: {}".format(
            time, len(remaining_cars), exited_count,
            [round(car.x, 3) for car in remaining_cars],
            [round(car.v, 3) for car in remaining_cars]
        ))

In [20]:
import random

class TestRoad(unittest.TestCase):
    '''
    A class to test the road simulation.
    '''

    def test_road_simulation(self):
        '''
        Test the simulation of cars on a road.
        '''
        road_length = 5
        initial_cars = [Car(105, 0.001, 2.05, 60), Car(105, 0.001, 2.04, 70)]
        road = Road(length=road_length, initial_cars=initial_cars)
        road.simulate(duration=10)

        for car in road.cars:
            # Check that no car has a negative speed
            self.assertGreaterEqual(car.v, 0)

            # Check that no car is crossing the length of the road
            self.assertLessEqual(car.x, road_length)

if __name__ == '__main__': #modified to run only 'TestRoad.test_road_simulation'
    unittest.main(argv=['first-arg-is-ignored', '-k', 'TestRoad.test_road_simulation'], exit=False) 

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Time: 0s - Number of Cars: 2 - Exited Cars: 0 - Car Positions: [2.079, 2.049] - Car Speeds: [105, 32.4]
Time: 1s - Number of Cars: 2 - Exited Cars: 0 - Car Positions: [2.108, 2.078] - Car Speeds: [105, 105]
New car added at time 2s with speed 118.894.
Time: 2s - Number of Cars: 3 - Exited Cars: 0 - Car Positions: [2.138, 2.107, 0.029] - Car Speeds: [105, 105, 105]
Time: 3s - Number of Cars: 3 - Exited Cars: 0 - Car Positions: [2.167, 2.137, 0.058] - Car Speeds: [105, 105, 105]
Time: 4s - Number of Cars: 3 - Exited Cars: 0 - Car Positions: [2.196, 2.166, 0.087] - Car Speeds: [105, 105, 105]
Time: 5s - Number of Cars: 3 - Exited Cars: 0 - Car Positions: [2.225, 2.195, 0.117] - Car Speeds: [105, 105, 105]
New car added at time 6s with speed 69.117.
Time: 6s - Number of Cars: 4 - Exited Cars: 0 - Car Positions: [2.254, 2.224, 0.146, 0.029] - Car Speeds: [105, 105, 105, 105]
New car added at time 7s with speed 108.928.
Time: 7s - Number of Cars: 5 - Exited Cars: 0 - Car Positions: [2.283, 2

In [ ]:
class Road:
    def __init__(self, length, initial_cars=[]):
        self.length = length
        self.cars = initial_cars
    def add_car(self, initial_speed):
        new_car = Car(105, 0.001, 0, initial_speed)
        self.cars.append(new_car)
    def update_cars(self, dt):
        exited_cars_count = 0
        for i in range(len(self.cars) - 1, -1, -1):
            front_car = self.cars[i - 1] if i > 0 else None
            self.cars[i].update_velocity(front_car)
            self.cars[i].update_position()
            if self.cars[i].x >= self.length:
                self.cars.pop(i)
                exited_cars_count += 1
        if exited_cars_count > 0:
            print("{} car(s) exited the road.".format(exited_cars_count))
        return exited_cars_count
    def simulate(self, duration):
        for t in range(11):  # 10 iterations
            probability = random.random()
            if probability == 0.5:
                initial_speed = random.uniform(50, 120)
                self.add_car(initial_speed)
                print("New car added at time {}s with speed {}.".format(round(t, 3), round(initial_speed, 3)))
            exited_count = self.update_cars(dt=1)
            self.print_state(t, exited_count)
    def print_state(self, time, exited_count):
        remaining_cars = [car for car in self.cars if car.x >= 0 and car.x <= self.length]
        print("Time: {}s - Number of Cars: {} - Exited Cars: {} - Car Positions: {} - Car Speeds: {}".format(
            time, len(remaining_cars), exited_count,
            [round(car.x, 3) for car in remaining_cars],
            [round(car.v, 3) for car in remaining_cars]
        ))